In [1]:
from dataset import get_path, get_subjects, epoch_data, epoch_runs
from utils import decod, correlate, match_list, create_target, analysis

In [2]:
import mne_bids
from pathlib import Path
import pandas as pd
import numpy as np
import mne
import spacy
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import RidgeCV
from wordfreq import zipf_frequency
from Levenshtein import editops
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use("Agg")
mne.set_log_level(False)

In [3]:
decoding_criterion = "word_length"

if decoding_criterion == "embeddings":
    nlp = spacy.load("fr_core_news_sm")


report = mne.Report()
path = get_path("LPP_read")
print(path)
subjects = get_subjects(path)
RUN = 2
task = "read"

print("\nSubjects for which the decoding will be tested: \n")
print(subjects)
subject = subjects[1]

/home/co/data/BIDS_lecture

Subjects for which the decoding will be tested: 

['1', '2', '3', '4', '5', '6']


In [4]:
epochs = epoch_runs(subject, RUN, task, path)

.Running the script on RAW data:
run 01, subject: 2


/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: Omitted 117 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015

['lorsque' 'javais' 'six' ... 'pas' 'aller' 'bien'] ['lorsque' 'j' 'avais' ... 'aller' 'bien' 'loin..."']


/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:124: RuntimeWarning: The measurement information indicates a low-pass frequency of 20.0 Hz. The decim=20 parameter will result in a sampling frequency of 50.0 Hz, which can cause aliasing artifacts.
  epochs = mne.Epochs(raw, **mne_events(meta, raw), decim=20, tmin=-0.2, tmax=0.8)


.Running the script on RAW data:
run 02, subject: 2


/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: Omitted 146 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:63: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015

["j'avais" 'ainsi' 'appris' ... 'ne' 'répondit' 'pas.'] ['j' 'avais' 'ainsi' ... 'ne' 'répondit' 'pas.']


/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:124: RuntimeWarning: The measurement information indicates a low-pass frequency of 20.0 Hz. The decim=20 parameter will result in a sampling frequency of 50.0 Hz, which can cause aliasing artifacts.
  epochs = mne.Epochs(raw, **mne_events(meta, raw), decim=20, tmin=-0.2, tmax=0.8)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:298: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs)


In [15]:
scores = dict()
scores['n_closing'] = decod(epochs, 'n_closing')

  0%|          | 0/51 [00:00<?, ?it/s]

In [43]:
plt.clf()
times = np.linspace(-.2, 0.8, scores['n_closing'].shape[0])
plt.fill_between(times, (scores[key]), label=key)
plt.savefig('./test_n_closing.jpeg')